In [ ]:
import time

import Libs.Adb as adb
from Libs.Beholder import *
from Libs.StateMachine import StateMachine

In [ ]:
import math

In [ ]:
adb.run_command(
    "shell am start com.jagex.runescape.android/com.jagex.android.MainActivity"
)

In [ ]:
gen = adb.ImageSource()
touch = adb.Toucher(gen)
a = Beholder(ImageSource=gen, toucher=touch)

a.addLayerModifer(Beholder_Layer_Chopper_Grayscale(from_layer="image", enabled=True))
a.addLayerModifer(
    Beholder_Layer_Chopper_AtCord(
        name="inventory_color",
        from_layer="image",
        x=1903,
        y=355,
        w=2222 - 1903,
        h=905 - 355,
        enabled=True,
    )
)
a.addLayerModifer(
    Beholder_Layer_Chopper_AtCord(
        name="inventory",
        from_layer="image_gray",
        x=1903,
        y=355,
        w=2222 - 1903,
        h=905 - 355,
        enabled=True,
    )
)
a.addLayerModifer(
    Beholder_Layer_Chopper_AtCord(
        name="Horizon",
        from_layer="image",
        x=690,
        y=320,
        w=1870 - 690,
        h=1,
        enabled=False,
    )
)

a.addLayerModifer(
    Beholder_Layer_Chopper_InRange(
        name="Horizon Enemy Finder",
        from_layer="Horizon",
        lbounds=[3, 0, 0],
        ubounds=[13, 248, 255],
        enabled=False,
    )
)

a.addLayerModifer(
    Beholder_Layer_Chopper_PeakExtractor(
        name="Horizon Enemy Finder Peaks",
        from_layer="Horizon Enemy Finder",
        threshhold=5,
        avg_fwd=5,
        enabled=False,
    )
)
adb.run_command("wait-for-device")

In [ ]:
def withdrawlBankItem(text, count=10):
    a.tap(2010, 130)
    time.sleep(0.3)
    a.tap(2140, 240)
    time.sleep(1)
    a.type(count)
    time.sleep(0.2)
    a.tap(1272, 130)
    time.sleep(0.2)
    a.tap(190, 200)
    time.sleep(0.2)
    a.tap(737, 130)
    time.sleep(1)
    a.type(text)
    time.sleep(0.5)
    a.tap(188, 288)
    time.sleep(0.5)
    a.tap(188, 288)
    time.sleep(0.5)

In [ ]:
home_button = (189, 89)
three_dots = (180, 80)
lobby = (320, 890)
favorite = []
favorite.append((1784, 1050))
favorite.append((1890, 1050))
favorite.append((1996, 1050))
world_index = 0


def switchWorlds(world_index):
    wait_time = 2
    long_wait = 7
    extra_long_wait = 14

    a.tap(*home_button)
    time.sleep(wait_time)

    a.tap(*three_dots)
    time.sleep(wait_time)

    a.tap(*lobby)
    time.sleep(long_wait)

    a.tap(*favorite[world_index])
    time.sleep(extra_long_wait)

In [ ]:
lodestone_matcher = Beholder_Image_Matcher(
    name="lodestone",
    layer="image_gray",
    filename="./rs3_templates/map/lodestone.png",
    convertToGray=True,
)
ui_yes_matcher = Beholder_Image_Matcher(
    name="ui_yes",
    layer="image_gray",
    filename="./rs3_templates/ui/Yes.png",
    convertToGray=True,
)


def teleportToNamedLocation(name):
    a.tap(2322, 157)
    time.sleep(1)
    a.tap(330, 85)
    a.type(name)
    time.sleep(1)
    matches = a.findMatches({"lodestone": lodestone_matcher})
    if "lodestones" in matches:
        lodestones = matches["lodestone"]

        center_x, center_y = [int(ele / 2) for ele in a.layers["image_gray"].data.shape]

        dist_eles = {}
        for stone in lodestones:
            stone_x, stone_y = stone.center
            dist = math.sqrt(
                math.pow(center_x - stone_x, 2) + math.pow(center_y - stone_y, 2)
            )
            dist_eles[stone] = dist
        print(dist_eles)
        s = sorted(dist_eles, key=lambda a: dist_eles[a])
        a.tap(*s[0].center)
        time.sleep(1)
        matches = a.findMatches({"ui_yes": ui_yes_matcher})
        if "ui_yes" in matches:
            ui_yes = matches["ui_yes"]
            a.tap(*ui_yes[0].center)
    time.sleep(1)

In [ ]:
a.findMatches({})

In [ ]:
raise Exception()

# Auto-Repeat smithing -stable


In [ ]:
a.matchers = {}
a.addMatcher(
    Beholder_Image_Matcher(
        name="Button",
        layer="image_gray",
        filename="./rs3_templates/ui/Begin_project.png",
        convertToGray=True,
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="UnfinishedSmithing",
        layer="inventory",
        filename="./rs3_templates/inventory/forge_item.png",
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="FinishedArrow",
        layer="image_gray",
        filename="./rs3_templates/ui/FinishedArrow.png",
        convertToGray=True,
    )
)

upgrade_level_pos = [
    (1532, 223),
    (1623, 223),
    (1700, 223),
    (1770, 223),
    (1840, 223),
    (1623, 223),
    (1920, 223),
]


def ChangeUpgradeLevel(level):
    global upgrade_old_level
    if upgrade_say_level_change:
        if level != upgrade_old_level:
            upgrade_old_level = level
            tts(f"Changing Upgrade level to {level}")
    center = upgrade_level_pos[level]
    adb.run_command(f"shell input tap {center[0]} {center[1]}")

In [ ]:
upgrade_say_level_change = True
upgrade_selected_item = True
upgrade_start_level = 2
upgrade_max_level = 4
upgrade_old_level = None
say_heat = False

upgrade_current_level = upgrade_start_level
time_since_heat = time.time()
time_smithing = time.time()
heating_timer = 30

In [ ]:
Button_count = 0
Anvil_count = 0
tts("starting autosmith")
while True:
    matches = a.findMatches()
    if len(matches) > 0:
        if "UnfinishedSmithing" in matches:
            Button_count = 0
            Anvil_count = 0
            if time.time() - time_since_heat > heating_timer:
                a.tap(1192, 150)
                if say_heat:
                    tts("heat", volume=0.1)
                sleep(3)
                a.tap(1550, 525)
                time_since_heat = time.time()
            sleep(7)
            continue
        if "FinishedArrow" in matches:
            center = matches["FinishedArrow"][0].center
            a.tap(center[0], center[1])
            tts(
                f"Completed in {secondsToTextDescription(int(time.time()-time_smithing))} "
            )
            time_smithing = time.time()
            sleep(1)
            continue
        if "Button" in matches:
            if Button_count == 0:

                ChangeUpgradeLevel(upgrade_current_level)
                center = matches["Button"][0].center
                a.tap(center[0], center[1])
                # tts("starting item")
                Button_count += 1
                sleep(2)
            elif Button_count > 5:
                tts("Cannot continue smithing. Possible Out of ores")
                a.layers["image"].show()
                break
            else:
                if not upgrade_selected_item:
                    tts("Finished, Please select new item you have 20 seconds")
                    time.sleep(20)
                else:
                    if (upgrade_current_level + 1) <= upgrade_max_level:
                        upgrade_current_level += 1
                        tts(
                            f"Next Upgrade Level {upgrade_current_level} out of {upgrade_max_level}"
                        )
                        ChangeUpgradeLevel(upgrade_current_level)
                        sleep(2)
                        Button_count = 0
                    else:
                        tts("Final Level Completed")
                        break

                center = matches["Button"][0].center
                a.tap(center[0], center[1])
                Button_count += 1
                sleep(4)

    else:
        if Anvil_count > 5:
            tts("Cannot continue smithing. Possible Inventory Scanning Issue")
            a.layers["image"].show()
            break
        if Anvil_count > 0:
            tts("Please restore game screen, cannot find inventory")
            sleep(7)
        else:
            a.tap(1550, 525)
            time_since_heat = 0
            Anvil_count += 1
    sleep(3.75)
tts("Cannot Continue")

In [ ]:
tts("finishing current smith")
while True:
    matches = a.findMatches()
    if len(matches) > 0:
        if "UnfinishedSmithing" not in matches:
            tts(
                f"Completed in {secondsToTextDescription(int(time.time()-time_smithing))} "
            )
            break
        else:
            if time.time() - time_since_heat > heating_timer:
                a.tap(1192, 150)
                if say_heat:
                    tts("heat", volume=0.1)
                sleep(5)
                adb.run_command(f"shell input tap 1550 525")
                time_since_heat = time.time()
    sleep(10)
tts("finished please come do something")

# Auto Smelt Ores -stable

In [ ]:
a.matchers = {}

a.addMatcher(
    Beholder_Image_Matcher(
        name="close_btb",
        layer="image_gray",
        filename="./rs3_templates/ui/crafting_dialog_close_button.png",
        convertToGray=True,
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="Button",
        layer="image_gray",
        filename="./rs3_templates/ui/Begin_project.png",
        convertToGray=True,
    )
)

In [ ]:
tts("starting auto smelt")
s = time.time()
Button_count = 0
while True:
    matches = a.findMatches()
    if len(matches) > 0:
        if "close_btb" in matches:
            Button_count = 0
            sleep(5)
            continue
        else:
            if "Button" in matches:
                if Button_count == 0:
                    center = matches["Button"][0].center
                    a.tap(center[0], center[1])

                    Button_count += 1
                    s = time.time()
                if Button_count == 2:
                    tts("Out of Ore Please select new ore type")
                    a.layers["image"].show()
                    sleep(15)
                if Button_count == 3:
                    break
                else:
                    a.tap(518, 1029)
                    Button_count += 1
    else:
        tts(f"Fin, {int(time.time()-s)} Seconds.")
        a.tap(1150, 780)
        sleep(3)
        a.tap(518, 1029)
    sleep(3)
tts("Finished")

# Bolt Featherer -stable


In [ ]:
a.matchers = {}
a.addMatcher(
    Beholder_Image_Matcher(
        name="bolts",
        layer="gray",
        filename="./templates/inventory/addy_bolts.png",
        convertToGray=True,
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="fletch",
        layer="gray",
        filename="./templates/ui/fletch.png",
        convertToGray=True,
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="close_btb",
        layer="gray",
        filename="./templates/ui/crafting_dialog_close_button.png",
        convertToGray=True,
    )
)

in_menu = False
while True:
    matches = a.findMatches()
    if len(matches) > 0:
        if "close_btb" in matches:
            print("Currently Crafting")
            sleep(2)
            continue
        if in_menu:
            if "fletch" in matches:
                center = matches["fletch"][0].center
                adb(f"shell input tap {center[0]} {center[1]}")
                in_menu = False
        else:
            if "bolts" in matches:
                center = matches["bolts"][0].center
                adb(f"shell input tap {center[0]} {center[1]}")
                in_menu = True
    else:
        break
    sleep(2)
tts("Cannot Continue")

a.layers["image"].show()

# Mine and store in OreChest/NotePaper/Mining Stamina -stable


In [ ]:
a.matchers = {}
a.addMatcher(
    Beholder_Image_Matcher(
        name="paper",
        layer="image_gray",
        filename="./rs3_templates/inventory/notepaper.png",
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="ore",
        layer="image_gray",
        filename="./rs3_templates/inventory/ore/drak.png",
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="orebox",
        layer="image_gray",
        filename="./rs3_templates/inventory/ore_boxes/bane.png",
    )
)

s = time.time()
ore_box_full = False
start_mining = 0
total_mined = 22
mine_at_least = 2000
previous_ore_count = 0

In [ ]:
log_ores = False
log_resets = False
say_method = False
reset_mining_timeout = 15

In [ ]:
if total_mined > 0:
    tts(f"resuming AutoMiner {total_mined} out of {mine_at_least}")
else:
    tts("starting AutoMiner")
while True:
    if time.time() - start_mining >= reset_mining_timeout:
        if log_resets:
            print("reset mining")
        adb.run_command(f"shell input tap 1215 830")
        start_mining = time.time()
    matches = a.findMatches()
    if len(matches.keys()) > 0:
        if "ore" in matches:
            if log_ores:
                if previous_ore_count != len(matches["ore"]):
                    print(f"{len(matches['ore'])} ores in inventory")
                    previous_ore_count = len(matches["ore"])
            if len(matches["ore"]) >= 10:
                total_mined += len(matches["ore"])
                tts(
                    f"took {secondsToTextDescription(int(time.time()-s))}, {total_mined} Total"
                )
                s = time.time()

                if mine_at_least < total_mined:
                    tts(f"finished aquiring {mine_at_least} ore")
                    break
                if not ore_box_full:
                    if "orebox" in matches:
                        if say_method:
                            tts("oreboxing")
                        center = matches["orebox"][0].center
                        adb.run_command(f"shell input tap {center[0]} {center[1]}")
                        time.sleep(5)
                        matches = a.findMatches()
                        if "ore" in matches.keys():
                            ore_box_full = True
                            tts("orebox is full, starting to use paper")

                        else:
                            start_mining = 0
                            continue

                if "ore" in matches and len(matches["ore"]) > 1:
                    if "paper" in matches:
                        if say_method:
                            tts("papering")
                        center = matches["ore"][0].center
                        adb.run_command(f"shell input tap {center[0]} {center[1]}")
                        time.sleep(0.5)
                        center = matches["paper"][0].center
                        adb.run_command(f"shell input tap {center[0]} {center[1]}")
                        start_mining = 0
                        time.sleep(2)
                    else:
                        tts("no more paper")
                        break

    if start_mining != 0:
        sleep(10)
tts("Exiting")
a.layers["inventory"].show()

# Scareb gather -wip

In [ ]:
a.matchers = {}
for i in range(0, 5):
    print(i)
    a.addMatcher(
        Beholder_Image_Matcher(
            name="bug",
            layer="gray",
            filename=f"./rs3_templates/bug{i}.png",
            convertToGray=True,
            threshhold=0.9,
        )
    )
while True:
    matches = a.findMatches()
    if len(matches) > 0:
        print(matches)
    print(".")
tts("Cannot Continue")

# Enemy Attacker -wip

In [ ]:
a.layer_modifiers["Horizon"].enabled = True
a.layer_modifiers["Horizon Enemy Finder"].enabled = True
a.layer_modifiers["Horizon Enemy Finder Peaks"].enabled = True

In [ ]:
while True:
    a.readNextImage()
    a.digestImage()
    item = max(a.layers["Horizon Enemy Finder Peaks"], key=lambda e: e[1] - e[0])
    print(item)
    center = (int(item[1] / 2 + item[0] / 2), 360)
    print(center)
    adb(f"shell input tap {center[0]} {center[1]}")
    sleep(5)

In [ ]:
a.layers

# Trout Fisher

In [ ]:
a.matchers = {}


a.addMatcher(
    Beholder_Image_Matcher(
        name="fish",
        layer="inventory",
        filename="./rs3_templates/fishing/fish.png",
        convertToGray=True,
        threshhold=0.95,
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="button",
        layer="image_gray",
        filename="./rs3_templates/ui/Next.png",
        convertToGray=True,
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="paper",
        layer="inventory",
        filename="./rs3_templates/inventory/notepaper.png",
        convertToGray=True,
    )
)
paper_time = time.time()
paper_time_timeout = 30
fishing_started = time.time()
storage_mode = False
matches = a.findMatches()
print({x: len(matches[x]) for x in matches.keys()})

In [ ]:
while True:
    matches = a.findMatches()
    print({x: len(matches[x]) for x in matches.keys()})
    if not storage_mode:
        if "button" in matches:
            center = matches["button"][0].center
            adb.run_command(f"shell input tap {center[0]} {center[1]}")
            tts(f"Completed in {int(time.time()-fishing_started)} seconds")
            fishing_started = time.time()
            print("start storage")
            storage_mode = True
        else:
            sleep(10)
    else:
        persist = False
        if "paper" not in matches:
            tts("No paper")
            break
        for _ in range(4):
            if "fish" in matches:
                print("storeing paper")
                center = matches["paper"][0].center
                adb.run_command(f"shell input tap {center[0]} {center[1]}")
                time.sleep(1)
                center = matches["fish"][0].center
                adb.run_command(f"shell input tap {center[0]} {center[1]}")
                print({x: len(matches[x]) for x in matches.keys()})
                time.sleep(1)
                matches = a.findMatches()
            storage_mode = False
        time.sleep(0.7)
        adb.run_command(f"shell input tap 1150 780")
        time.sleep(0.7)

# Tree Cutter

In [ ]:
a.matchers = {}
a.addMatcher(
    Beholder_Image_Matcher(
        name="paper",
        layer="inventory",
        filename="./rs3_templates/inventory/notepaper.png",
        convertToGray=True,
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="logs",
        layer="inventory",
        filename="./rs3_templates/inventory/logs/Achey.png",
        convertToGray=True,
    )
)
tree_locations = {
    "up": False,
    "right": False,
    "down": False,
    "left": True,
}
click_loc_pos = {
    "up": "1192 150",
    "right": "1550 525",
    "down": "1192 830",
    "left": "755 525",
}
tree_regrow_timer = 3
start_loggin = time.time()

In [ ]:
tree_regrow_timer_each = tree_regrow_timer / sum(
    [1 if tree_locations[a] else 0 for a in tree_locations]
)
tree_regrow_timer_each

In [ ]:
while True:
    for loc in ["up", "right", "down", "left"]:
        if tree_locations[loc]:
            adb.run_command(f"shell input tap {click_loc_pos[loc]}")
            sleep(tree_regrow_timer_each)
    matches = a.findMatches()
    if "logs" in matches:
        if len(matches["logs"]) >= 10:
            tts(f"cten {int(time.time()-start_loggin)}")
            start_loggin = time.time()
            center = matches["paper"][0].center
            adb.run_command(f"shell input tap {center[0]} {center[1]}")
            time.sleep(1)
            center = matches["logs"][0].center
            adb.run_command(f"shell input tap {center[0]} {center[1]}")
    sleep(2)

# repeat crafting - stable

In [ ]:
a.matchers = {}
a.addMatcher(
    Beholder_Image_Matcher(
        name="close_btb",
        layer="image_gray",
        filename="./rs3_templates/ui/crafting_dialog_close_button2.png",
    )
)
start = time.time()
in_menu = False
auto_reset_timer = 180
min_cutoff_kill_timeout = 8

In [ ]:
while True:
    matches = a.findMatches()
    if (
        "close_btb" in matches
        and not in_menu
        and not time.time() - start > auto_reset_timer
    ):
        print(f"Currently Processsing { time.time() - start}")
        sleep(5)
        continue
    else:
        if in_menu:
            adb.run_command(f"shell input tap 1560 977")
            in_menu = False
            sleep(2.5)
        else:
            adb.run_command(f"shell input tap 1936 386")
            start = time.time()
            in_menu = True
            sleep(0.5)
tts("Cannot Continue")

a.layers["image"].show()

# Directional Clicker -beta


In [ ]:
click_locations = {
    "up": False,
    "right": False,
    "down": False,
    "left": True,
}
click_loc_pos = {
    "up": "1192 150",
    "right": "1550 525",
    "down": "1192 830",
    "left": "755 525",
}
click_delay = 15
while True:
    for loc in ["up", "right", "down", "left"]:
        if click_locations[loc]:
            adb.run_command(f"shell input tap {click_loc_pos[loc]}")
            sleep(click_delay)
            tts("click")

# Telekinetic grab training -wip -dumbclicker

In [ ]:
withdrawlBankItem("Law rune", 100)
withdrawlBankItem("Air rune", 100)
print("goto varrock west bank and stand in the east door gate")

In [ ]:
necklace = (1066, 965)
bar = (1196, 760)

magic_book = (2330, 690)
skill_page = (2114, 315)
grab_spell = (2014, 551)


backpack = (2330, 326)

In [ ]:
a.matchers = {}
a.addMatcher(
    Beholder_Image_Matcher(
        name="paper",
        layer="inventory",
        filename="./rs3_templates/inventory/notepaper.png",
        convertToGray=True,
    )
)

In [ ]:
while True:
    a.tap(*grab_spell)
    time.sleep(2.5)
    a.tap(*bar)
    time.sleep(2.5)

    a.tap(*grab_spell)
    time.sleep(1.5)
    a.tap(*necklace)

    sleep(5)
    a.tap(*backpack)
    matches = a.findMatches()
    if "gold" in matches:
        print("found things")
    if "necklace" in matches:
        print("found things")

    sleep(5)
    switchWorlds(world_index % len(favorite))
    world_index += 1
    sleep(7)
    a.tap(*magic_book)
    time.sleep(2.5)
    a.tap(*skill_page)
    time.sleep(3)

# Protean Bars

In [ ]:
a.matchers = {}
a.addMatcher(
    Beholder_Image_Matcher(
        name="close_btb",
        layer="image_gray",
        filename="./rs3_templates/ui/crafting_dialog_close_button2.png",
    )
)

auto_reset_timer = 480
down = "1192 830"
start = time.time()
in_menu = False

In [ ]:
while True:
    matches = a.findMatches()
    if (
        "close_btb" in matches
        and not in_menu
        and not time.time() - start > auto_reset_timer
    ):
        print(f"Currently Processsing { time.time() - start}")
        sleep(5)
        continue
    else:
        if in_menu:
            adb.run_command(f"shell input tap 1560 977")
            in_menu = False
            sleep(2.5)
        else:
            adb.run_command(f"shell input tap {down}")
            start = time.time()
            in_menu = True
            sleep(0.5)
tts("Cannot Continue")

a.layers["image"].show()

# Steal Silver

In [ ]:
a.matchers = {}
a.addMatcher(
    Beholder_Image_Matcher(
        name="silver",
        layer="image_gray",
        filename="./rs3_templates/theifing/silverbar.png",
    )
)
start = time.time()
stolen = 2
attacked = 0
attempt = 0
startup = True

In [ ]:
previous_detect = False
while True:
    matches = a.findMatches()
    if "silver" in matches:
        center = matches["silver"][0].center
        a.tap(center[0], center[1])
        if attempt == 1:
            if time.time() - start < 10:
                attacked += 1
                attempt = 0
                tts(f"attacked")
                sleep(10)
        else:
            attempt += 1
            previous_detect = True

        start = time.time()
    else:
        attempt = 0
        if previous_detect == True:
            stolen += 1
            attempt = 0
            tts(f"{stolen} taken")
            previous_detect = False
    sleep(2)
tts("Cannot Continue")

a.layers["image"].show()

In [ ]:
stolen, attacked

# Random Stuff - wip


In [ ]:
s = time.time()

In [ ]:
time.time() - s

# Other Stuff

In [ ]:
matches = a.findMatches()
print({x: len(matches[x]) for x in matches.keys()})

In [ ]:
time.time() - s

In [ ]:
for index in range(0, 100):
    print(
        index,
    )

In [ ]:
tts("Cannot Continue")

In [ ]:
a.layers

In [ ]:
a.layer_modifiers["Horizon"].enabled = True
a.layer_modifiers["Horizon Enemy Finder"].enabled = True

In [ ]:
target_color = (143, 4, 13)
tolarance = 75

target_info_cords = (460, 546)
target_info_color = (17, 42, 50)

In [ ]:
a.layers["Horizon Enemy Finder"].show()

In [ ]:
a.layers["Horizon Enemy Finder"].show()

In [ ]:
for item in a.matchers:
    print(item.name, item.data.shape)

In [ ]:
matches["Salmon"][0].show(a)

In [ ]:
a.layers["image_gray"].data.shape